# Exploración del Sistema de Clasificación Documental Guiada (RAG)

Este notebook te permitirá explorar y entender los componentes principales del sistema RAG para la clasificación de capturas de pantalla según lineamientos en documentos PDF.

In [ ]:
# Importaciones necesarias
import sys
import os
from pathlib import Path

# Asegurarnos que el módulo principal sea importable
sys.path.append(os.path.abspath('..')) 

# Importar componentes
from app.ocr import OCREngine, extract_text_from_image
from app.pdf_parser import PDFParser, extract_pdf_content
from app.vector_store import VectorStore, create_vector_store_from_pdf
from app.rag_engine import LLMProvider, RAGEngine
from app.formatter import ResponseFormatter, format_llm_response

# Para visualizaciones
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np

## 1. OCR: Procesamiento de Imágenes

El primer paso es extraer el texto de una imagen o captura de pantalla utilizando OCR.

In [ ]:
# Configura la ruta a tu imagen de prueba
imagen_prueba = "../data/samples/captura_ejemplo.png"

# Visualizar la imagen
plt.figure(figsize=(10, 10))
plt.imshow(Image.open(imagen_prueba))
plt.axis('off')
plt.title('Imagen de entrada')
plt.show()

In [ ]:
# Inicializar motor OCR y extraer texto
ocr = OCREngine(engine="doctr")  # Opciones: "doctr" o "easyocr"

# Procesar imagen
resultado_ocr = ocr.process_image(imagen_prueba)

# Ver resultado
print(f"Texto extraído con confianza {resultado_ocr['confidence']:.2f}:\n")
print(resultado_ocr['text'])

In [ ]:
# Ver bloques de texto detectados
print(f"Bloques detectados: {len(resultado_ocr['blocks'])}")
for i, bloque in enumerate(resultado_ocr['blocks'][:5]):
    print(f"\nBloque {i+1}:")
    print(f"- Texto: {bloque['text'][:100]}..." if len(bloque['text']) > 100 else f"- Texto: {bloque['text']}")
    print(f"- BBox: {bloque['bbox']}")
    print(f"- Confianza: {bloque['confidence']:.2f}")

## 2. Procesamiento de PDF y Extracción de Texto

El segundo paso es procesar el documento PDF guía y extraer su contenido en chunks.

In [ ]:
# Configura la ruta a tu PDF de guía
pdf_guia = "../data/guidelines/guia_ejemplo.pdf"

# Procesar PDF
parser = PDFParser()
documento = parser.parse_pdf(pdf_guia)

# Ver información básica
print(f"Documento: {documento.title}")
print(f"Autor: {documento.author}")
print(f"Páginas: {documento.num_pages}")
print(f"Chunks generados: {len(documento.chunks)}")

In [ ]:
# Ver tabla de contenido
if documento.toc:
    print("Tabla de Contenido:")
    for item in documento.toc:
        nivel = item.get('level', 1)
        print(f"{' ' * (nivel-1)*2}{'#' * nivel} {item.get('title')} (pág. {item.get('page')})")

In [ ]:
# Explorar los primeros chunks
for i, chunk in enumerate(documento.chunks[:3]):
    print(f"\n--- Chunk {i+1} (Página {chunk.page_number}) ---")
    print(f"Sección: {chunk.section}")
    print(f"Subsección: {chunk.subsection}")
    print(f"Texto: {chunk.text[:200]}..." if len(chunk.text) > 200 else f"Texto: {chunk.text}")

## 3. Base de Datos Vectorial

El tercer paso es crear y consultar la base de datos vectorial.

In [ ]:
# Crear base de datos vectorial
vector_store_dir = "../vector_store"
vector_store, _ = create_vector_store_from_pdf(pdf_guia, vector_store_dir)

In [ ]:
# Ver estadísticas de la base de datos
stats = vector_store.get_collection_stats()
print(f"Nombre de colección: {stats.get('collection_name')}")
print(f"Total de chunks: {stats.get('count')}")

if 'pages' in stats:
    print("\nDistribución por páginas:")
    for page, count in sorted(stats['pages'].items()):
        print(f"- Página {page}: {count} chunks")

In [ ]:
# Probar una consulta semántica
consulta = "¿Cómo clasificar un caso donde aparece un pin mal ubicado en el mapa?"

resultados = vector_store.search(consulta, n_results=3)

print(f"Resultados para: '{consulta}'\n")
for i, resultado in enumerate(resultados):
    print(f"\n--- Resultado {i+1} ---")
    print(f"Score: {resultado['score']:.4f}")
    print(f"Página: {resultado['metadata'].get('page')}")
    print(f"Sección: {resultado['metadata'].get('section')}")
    print(f"Texto:\n{resultado['text'][:300]}..." if len(resultado['text']) > 300 else f"Texto:\n{resultado['text']}")

## 4. Motor RAG y LLM

El cuarto paso es integrar todo en el motor RAG con un LLM local para generar clasificaciones.

In [ ]:
# Verificar disponibilidad del modelo local (asegúrate de tener Ollama instalado)
try:
    llm_provider = LLMProvider(provider="ollama", model_name="mistral:7b")
    print("LLM inicializado correctamente")
except Exception as e:
    print(f"Error al inicializar LLM: {str(e)}")
    print("\nPara instalar Ollama: https://ollama.ai/")
    print("Para descargar el modelo: ollama pull mistral:7b")

In [ ]:
# Crear motor RAG
rag_engine = RAGEngine(
    vector_store=vector_store,
    llm_provider=llm_provider,
    ocr_engine=ocr
)

In [ ]:
# Procesar una imagen para clasificación
campos = ["Tipo de caso", "Prioridad", "Área responsable"]

# Este proceso puede tardar dependiendo del tamaño de la imagen y el modelo LLM
resultado = rag_engine.process_image(
    image_path=imagen_prueba,
    fields_to_classify=campos,
    n_results=5
)

In [ ]:
# Ver texto extraído
print("Texto extraído de la imagen:")
print(resultado["extracted_text"][:500] + "..." if len(resultado["extracted_text"]) > 500 else resultado["extracted_text"])

In [ ]:
# Ver clasificación generada
print("\nClasificación generada por el LLM:\n")
print(resultado["classification"])

## 5. Formateo de Resultados

Finalmente, formateamos los resultados en un JSON estructurado.

In [ ]:
# Formatear resultados
formatter = ResponseFormatter()
resultados_formateados = formatter.format_classification_response(resultado["classification"])

# Ver resultados formateados
import json
print(json.dumps(resultados_formateados, indent=2, ensure_ascii=False))

In [ ]:
# Ver clasificaciones en formato tabular
if "clasificaciones" in resultados_formateados:
    df_clasificaciones = pd.DataFrame(resultados_formateados["clasificaciones"])
    display(df_clasificaciones[["campo", "valor", "justificación"]])

## 6. Modo Chat con el Documento

También puedes consultar directamente el documento guía.

In [ ]:
# Realizar una consulta al documento
consulta = "¿Cuáles son los criterios para clasificar un caso como alta prioridad?"

respuesta = rag_engine.chat_with_document(consulta)
print(f"Consulta: {consulta}\n")
print(f"Respuesta: {respuesta.get('response')}")

In [ ]:
# Ver referencias del documento
print("Referencias del documento:")
for i, chunk in enumerate(respuesta.get("context_chunks", [])):
    print(f"\n--- Fuente {i+1} (Página {chunk.get('page', 'N/A')}) ---")
    print(chunk.get('text', ''))

## Conclusión

Este notebook muestra el flujo completo del sistema de clasificación documental guiada:

1. Extracción de texto con OCR
2. Procesamiento de PDF guía
3. Creación de base de datos vectorial
4. Recuperación aumentada con LLM
5. Generación de clasificaciones estructuradas

Para usar el sistema en producción, es recomendable utilizar la interfaz de línea de comandos implementada en `app/main.py`.